In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from fiona.crs import from_epsg
import geopandas as gpd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('requests.csv')

In [33]:
df.head()

,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff
0,1380949200,1149,725,218,3601
1,1380949200,1225,1714,2003,3601
2,1380949200,1237,3143,1314,2218
3,1380949200,1323,3566,1330,2573
4,1380949200,1328,2466,1323,2473


In [3]:
dropped = df[df['Actual_Pickup'] == -1]

In [4]:
manhattan = gpd.GeoDataFrame.from_file('manhattan.geojson')

In [5]:
dropped_gpd = manhattan.merge(dropped, left_on = 'id', right_on='Requested_Pickup')

In [6]:
dropped_gpd.head()

,id,w,geometry,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff
0,0,11,"LINESTRING (-73.9975944 40.7140611, -73.997492...",1380981268,0,1032,-1,-1
1,0,11,"LINESTRING (-73.9975944 40.7140611, -73.997492...",1380993780,0,3249,-1,-1
2,0,11,"LINESTRING (-73.9975944 40.7140611, -73.997492...",1380998460,0,402,-1,-1
3,0,11,"LINESTRING (-73.9975944 40.7140611, -73.997492...",1381016266,0,2851,-1,-1
4,0,11,"LINESTRING (-73.9975944 40.7140611, -73.997492...",1381026780,0,2769,-1,-1


In [10]:
ct_gpd = gpd.GeoDataFrame.from_file('man.geojson')

In [30]:
ct_gpd.ctlabel.unique().shape

(288,)

In [11]:
ct_gpd.to_crs(epsg=2263, inplace=True)

In [12]:
dropped_gpd.to_crs(epsg=2263, inplace=True)

In [13]:
sj_output = gpd.sjoin(ct_gpd,dropped_gpd,how='inner')

In [14]:
sj_output.head()

,ntacode,shape_area,ntaname,shape_leng,boro_name,puma,ct2010,boro_code,cdeligibil,boro_ct2010,ctlabel,geometry,index_right,id,w,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff
0,MN19,1906016.35002,Turtle Bay-East Midtown,5534.19981063,Manhattan,3808,009800,1,I,1009800,98,(POLYGON ((994133.5070190424 214848.8975829605...,54322,1168,213,1381027369,1168,4293,-1,-1
0,MN19,1906016.35002,Turtle Bay-East Midtown,5534.19981063,Manhattan,3808,009800,1,I,1009800,98,(POLYGON ((994133.5070190424 214848.8975829605...,54323,1168,213,1381027411,1168,4293,-1,-1
0,MN19,1906016.35002,Turtle Bay-East Midtown,5534.19981063,Manhattan,3808,009800,1,I,1009800,98,(POLYGON ((994133.5070190424 214848.8975829605...,54245,1168,213,1380980793,1168,236,-1,-1
0,MN19,1906016.35002,Turtle Bay-East Midtown,5534.19981063,Manhattan,3808,009800,1,I,1009800,98,(POLYGON ((994133.5070190424 214848.8975829605...,54182,1165,79,1381023960,1165,4109,-1,-1
96,MN19,1950021.49652,Turtle Bay-East Midtown,5627.47731022,Manhattan,3808,010800,1,I,1010800,108,(POLYGON ((994786.7404174833 216028.8560180134...,54182,1165,79,1381023960,1165,4109,-1,-1


In [15]:
ct_dropped = sj_output[['ctlabel', 'boro_code' ]]

In [16]:
dropped_rides_ct_count = ct_dropped.groupby('ctlabel').count()

In [17]:
dropped_rides_ct_count.rename(columns = {'boro_code' : 'dropped_rides_count'},inplace=True)

In [18]:
dropped_rides_ct_list = dropped_rides_ct_count.index.values.tolist()

In [19]:
total_cts_list = ct_gpd.ctlabel.unique().tolist()

In [20]:
excl_ct_ids = list(set(total_cts_list).difference(set(dropped_rides_ct_list)))

In [21]:
excl_ct_id_df = pd.DataFrame(excl_ct_ids,columns=['ctlabel'])

In [22]:
excl_ct_id_df['dropped_rides_count'] = 0
excl_ct_id_df.set_index('ctlabel',inplace=True)

In [23]:
total_cts_dropped = pd.concat([dropped_rides_ct_count,excl_ct_id_df],axis=0)

In [31]:
total_cts_dropped.to_json('ct_dropped.json')

In [25]:
total_cts_dropped

,dropped_rides_count
ctlabel,
10.01,635
10.02,575
100,1729
101,313
102,1852
103,1544
104,1202
106.01,501
106.02,500
